In [5]:
import pandas as pd
import numpy as np

In [6]:
data = {'organization_id': [1, 1, 0, 2, 3, 3, 4, 0, 4, 4],
        'customer_id': [1, 1, 1, 2, 3, 3, 4, 4, 4, 4],
        'summary_day': ['2015-04-20'] * 10,
        'event': ['add_t', 'add_t', 'drop_f', 'drop_f', 'drop_t', 'add_f', 'add_f', 'add_t', 'add_t', 'drop_t'],
        'score': [1] * 10,}

## groupby

In [7]:
df = pd.DataFrame(data)

groups = df.groupby(['summary_day', 'customer_id', 'organization_id', 'event'])
r = groups.aggregate(np.sum)
r.reset_index(inplace=True)
r

,summary_day,customer_id,organization_id,event,score
0,2015-04-20,1,0,drop_f,1
1,2015-04-20,1,1,add_t,2
2,2015-04-20,2,2,drop_f,1
3,2015-04-20,3,3,add_f,1
4,2015-04-20,3,3,drop_t,1
5,2015-04-20,4,0,add_t,1
6,2015-04-20,4,4,add_f,1
7,2015-04-20,4,4,add_t,1
8,2015-04-20,4,4,drop_t,1


In [8]:
def calc_sn_total(df):
    sn_add = [c for c in df.columns.tolist()
                if c.startswith('add')]
    sn_drop = [c for c in df.columns.tolist()
                if c.startswith('drop')]

    def calc(x):
        sum = 0
        for c in sn_add:
            sum += x[c]
        for c in sn_drop:
            sum -= x[c]
        return sum

    return calc


def assign_calc_columns(df):
    return df.assign(sn_total = calc_sn_total(df))

In [10]:
df = pd.DataFrame(data)
print df.head(n=10)

df['uniq'] = zip(df['customer_id'], df['organization_id'], df['summary_day'], df.index.values)
print ">>> uniq tuple added <<<"
print df.head(n=10)

df = df.pivot('uniq', 'event', 'score')
df.fillna(0, inplace=True)

s = [df['add_t'], df['add_f']]
df['add_total'] = np.add(*s)
print ">>> np <<<"
print df.head(n=10)

df = assign_calc_columns(df)

columns_score = df.columns.tolist()
columns_key = ['summary_day', 'customer_id', 'organization_id']

df['customer_id'] = [v[0] for v in df.index.values]
df['organization_id'] = [v[1] for v in df.index.values]
df['summary_day'] = [v[2] for v in df.index.values]
print ">>> add key columns back <<<"
print df.head(n=10)

columns_csv = columns_key + sorted(columns_score)
print ">>> csv columns <<<"
print columns_csv

df.to_csv("tmp.csv", columns=columns_csv, index=False)

   customer_id   event  organization_id  score summary_day
0            1   add_t                1      1  2015-04-20
1            1   add_t                1      1  2015-04-20
2            1  drop_f                0      1  2015-04-20
3            2  drop_f                2      1  2015-04-20
4            3  drop_t                3      1  2015-04-20
5            3   add_f                3      1  2015-04-20
6            4   add_f                4      1  2015-04-20
7            4   add_t                0      1  2015-04-20
8            4   add_t                4      1  2015-04-20
9            4  drop_t                4      1  2015-04-20
>>> uniq tuple added <<<
   customer_id   event  organization_id  score summary_day  \
0            1   add_t                1      1  2015-04-20   
1            1   add_t                1      1  2015-04-20   
2            1  drop_f                0      1  2015-04-20   
3            2  drop_f                2      1  2015-04-20   
4            3  

### pivot

In [11]:
df = pd.DataFrame(data)

df_pk = df[['summary_day', 'customer_id', 'organization_id']]

df_col = df[['event', 'score']]
df_col.index.name = 'pix'
df_col.reset_index(inplace=True)

df = df_col.pivot('pix', 'event', 'score')
df.fillna(0, inplace=True)

df =  pd.concat([df, df_pk], axis=1, join='inner')
df

event,add_f,add_t,drop_f,drop_t,summary_day,customer_id,organization_id
pix,,,,,,,
0,0,1,0,0,2015-04-20,1,1
1,0,1,0,0,2015-04-20,1,1
2,0,0,1,0,2015-04-20,1,0
3,0,0,1,0,2015-04-20,2,2
4,0,0,0,1,2015-04-20,3,3
5,1,0,0,0,2015-04-20,3,3
6,1,0,0,0,2015-04-20,4,4
7,0,1,0,0,2015-04-20,4,0
8,0,1,0,0,2015-04-20,4,4


## Blaze

In [12]:
a = [1,1]
b = [2,2]
c = [4,2]
d = [0,3]

m = np.array([a, b, c, d])
print m.sum(axis=0)

cast = pd.DataFrame([2.0, np.nan])
cast.fillna(-1, inplace=True)
print cast
#print cast.apply(lambda x: str(int(x)) )

[7 8]
   0
0  2
1 -1


In [13]:
from blaze import *

df = pd.DataFrame(data)
t = Data(df)

by([t.customer_id, t.organization_id], total = t.score.sum())

ImportError: No module named blaze